# Lambda - 1


In [ ]:
import json
import pytz
import boto3
import pandas as pd
from datetime import *
import random
import time
from io import StringIO
from botocore.exceptions import ClientError
  
SGT = pytz.timezone('Asia/Singapore')
bucket = 'ag-nonprd-spot-config'
sns_topic_arn = 'arn:aws:sns:ap-southeast-1:263975812908:Spot-Notification'

client = boto3.client('ec2')
ec2 = boto3.resource('ec2')
evnt_client = boto3.client('events')
lambda_client = boto3.client('lambda')
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
sns = boto3.client('sns')

def spot_init(InstanceType, iam_role_arn, iam_role_name, ami, key_pair_name, availability_zone, interuption_behavior, tags, eni):
    spot_create_response = client.request_spot_instances(
        InstanceCount=1,
        LaunchSpecification={
            'EbsOptimized': True,
            'IamInstanceProfile': {'Name': iam_role_name},
            'ImageId': ami,
            'InstanceType': InstanceType,
            'KeyName': key_pair_name,
            'Monitoring': {'Enabled': True}, 
            'NetworkInterfaces': [{'DeviceIndex': 0,'NetworkInterfaceId': eni}],
            'Placement': {'AvailabilityZone': availability_zone}},
        Type = 'persistent',
        TagSpecifications=[{'ResourceType': 'spot-instances-request', 'Tags': tags}],
        InstanceInterruptionBehavior = interuption_behavior
    )

    SpotInstanceRequestId = str(spot_create_response['SpotInstanceRequests'][0]['SpotInstanceRequestId'])
    print('Spot Instance Request_Id : ', SpotInstanceRequestId)

    time.sleep(2)
    request_response = client.describe_spot_instance_requests(SpotInstanceRequestIds = [SpotInstanceRequestId])
    while request_response['SpotInstanceRequests'][0]['State'] != 'active':
        request_response = client.describe_spot_instance_requests(SpotInstanceRequestIds = [SpotInstanceRequestId])
        if request_response['SpotInstanceRequests'][0]['State'] == 'failed':
            print('Requested Instance Type is not available')
            hunt_state = 'no'
            return [hunt_state, SpotInstanceRequestId, '']

    if request_response['SpotInstanceRequests'][0]['State'] == 'active':
        inst_id = request_response['SpotInstanceRequests'][0]['InstanceId']
        hunt_state = 'present'

        print('Spot Instance Request Status : ', request_response['SpotInstanceRequests'][0]['State'])
        print('Spot Instance ID : ', inst_id)

        instance_response = client.describe_instance_status(InstanceIds=[inst_id])
        while instance_response['InstanceStatuses'] == []:
            instance_response = client.describe_instance_status(InstanceIds=[inst_id])
        print('Instance Status : ', instance_response['InstanceStatuses'][0]['InstanceState']['Name'], 'and Initializing...')
        while instance_response['InstanceStatuses'][0]['InstanceStatus']['Details'][0]['Status'] != 'passed' and instance_response['InstanceStatuses'][0]['SystemStatus']['Details'][0]['Status'] != 'passed':
            instance_response = client.describe_instance_status(InstanceIds=[inst_id])
        print('Instance Status : started')
        ec2.create_tags(Resources=[inst_id], Tags = tags)
        print('Tag Status : updated')
        return [hunt_state, SpotInstanceRequestId , inst_id]

def stop_instance(inst_id):
    stop_response = client.stop_instances(InstanceIds=[inst_id])
    print('EC2 Instance temporarily stop initiated..')
    spot_inst_status = client.describe_instances(InstanceIds=[inst_id])
    while spot_inst_status['Reservations'][0]['Instances'][0]['State']['Name'] != 'stopped':
        spot_inst_status = client.describe_instances(InstanceIds=[inst_id])
    print('EC2 Instance temporarily stopped.')

def get_ebs_vol_id(inst_id):
    ebs_volume_response = client.describe_instance_attribute(InstanceId = inst_id, Attribute='blockDeviceMapping')
    while ebs_volume_response['BlockDeviceMappings'][0]['Ebs']['Status'] != 'attached':
        ebs_volume_response = client.describe_instance_attribute(InstanceId = inst_id, Attribute='blockDeviceMapping')
    ebs_vol_id = ebs_volume_response['BlockDeviceMappings'][0]['Ebs']['VolumeId']
    print('EBS Volume ID : ', ebs_vol_id)
    return ebs_vol_id
    
def volume_detach(inst_id, ebs_vol_id):
    vol_detach_response = client.detach_volume(Force = True, InstanceId = inst_id, VolumeId = ebs_vol_id)
    print('EC2 Instance volume is detached..')

    vol_status = client.describe_volumes(VolumeIds=[ebs_vol_id])
    while vol_status['Volumes'][0]['State'] != 'available':
        vol_status = client.describe_volumes(VolumeIds=[ebs_vol_id])

def attach_volume(inst_id, ebs_volums):
    vol_list = []
    for vol in ebs_volums:
        response = client.attach_volume(Device = vol['device_name'], InstanceId = inst_id, VolumeId = vol['volume_id'])
        vol_list.append(vol['volume_id'])

    vol_status = client.describe_volumes(VolumeIds = vol_list)
    while vol_status['Volumes'][0]['Attachments'][0]['State'] != 'attached' and vol_status['Volumes'][1]['Attachments'][0]['State'] != 'attached':
        vol_status = client.describe_volumes(VolumeIds = vol_list)
    print('Volumes attached sucessfully..')

def delete_volume(ebs_vol_id):
    response = client.delete_volume(VolumeId = ebs_vol_id)
    print('New EBS volume delete request cretaed sucessfully.')

def describe_spot_req(SpotInstanceRequestId):
        request_response = client.describe_spot_instance_requests(SpotInstanceRequestIds = [SpotInstanceRequestId])
        while request_response['SpotInstanceRequests'][0]['State'] == 'active':
            request_response = client.describe_spot_instance_requests(SpotInstanceRequestIds = [SpotInstanceRequestId])

def start_instance(inst_id):
    spot_start_response = client.start_instances(InstanceIds=[inst_id])
    print('EC2 Instance Start initiated..')
    spot_inst_status = client.describe_instance_status(InstanceIds=[inst_id])
    while spot_inst_status['InstanceStatuses'] == []:
        spot_inst_status = client.describe_instance_status(InstanceIds=[inst_id])
    print('EC2 Instance Status : ', spot_inst_status['InstanceStatuses'][0]['InstanceState']['Name'], 'and Initializing...')

def ondemand_init(virtual_name, ami, od_instance_type, availability_zone, key_pair_name, iam_role_arn, iam_role_name, tags, eni):
    instance_create_respopnse = ec2.create_instances(
        BlockDeviceMappings=[
            {
                'DeviceName': '/dev/sda1',
                'VirtualName': virtual_name,
                'Ebs': {'DeleteOnTermination': False, 'VolumeSize': 30, 'VolumeType': 'standard'}
            },
        ],
        ImageId = ami,
        InstanceType = od_instance_type,
        KeyName = key_pair_name,
        Monitoring={'Enabled': True},
        MaxCount=1,
        MinCount=1,
        Placement={'AvailabilityZone': availability_zone},
        EbsOptimized=True,
        IamInstanceProfile = {'Name': iam_role_name},
        NetworkInterfaces=[{'DeviceIndex': 0, 'NetworkInterfaceId': eni}],
        InstanceInitiatedShutdownBehavior='stop',
        TagSpecifications=[{'ResourceType': 'instance','Tags': tags}]
        )
        
    inst_id = str(instance_create_respopnse[0]).split("'")[1]
    print('On-Demand Instance ID : ', inst_id)
    instance_status = client.describe_instance_status(InstanceIds=[inst_id])
    while instance_status['InstanceStatuses'] == []:
        instance_status = client.describe_instance_status(InstanceIds=[inst_id])
    print('On-Demand Instance Status : ', instance_status['InstanceStatuses'][0]['InstanceState']['Name'], 'and Initializing...')
    while instance_status['InstanceStatuses'][0]['InstanceStatus']['Details'][0]['Status'] != 'passed' and instance_status['InstanceStatuses'][0]['SystemStatus']['Details'][0]['Status'] != 'passed':
        instance_status = client.describe_instance_status(InstanceIds=[inst_id])
    print('On-Demand Instance Status : started')
    ec2.create_tags(Resources=[inst_id], Tags = tags)
    print('Tag Status : updated')
    return inst_id
    
def create_eventrule(rulename, lambda_RoleARN, inst_id):
    put_rule_response = evnt_client.put_rule(
        Name = rulename,
        ScheduleExpression = "rate(4 hours)",
        State = 'DISABLED'
    )
    rule_arn = put_rule_response['RuleArn']
    print('Rule is created, Rule Name : ', rulename)

    sts_inst_id = '{"instance_id":' + '"' + inst_id + '"' + '}'
    target_response = evnt_client.put_targets(
        Rule = rulename,
        Targets = [{'Arn': lambda_RoleARN,
                    'Id': 'abc',
                    'Input':sts_inst_id}])
                    
    print('Target is attached to rule.')

    enable_response = evnt_client.enable_rule(Name = rulename)
    print('Rule is Enabled sucessfully.')

    lambda_add_per_res = lambda_client.add_permission(FunctionName = lambda_RoleARN.split(':')[-1], StatementId=str(random.randint(10000,50000)), Action='lambda:InvokeFunction', Principal='events.amazonaws.com', SourceArn=rule_arn)
    print('Lambda trigger permission added sucessfully.')

def cancel_spot_request(spot_request_id):
    cancel_spot_request_response = client.cancel_spot_instance_requests(SpotInstanceRequestIds=[spot_request_id])

def terminate_instances(instance_id, start_datetime, log_file_dict, app_name, bucket):
    response = client.terminate_instances(InstanceIds=[instance_id])
    inst_status = client.describe_instances(InstanceIds=[instance_id])
    while inst_status['Reservations'][0]['Instances'][0]['State']['Name'] != 'terminated':
        inst_status = client.describe_instances(InstanceIds=[instance_id])
    for log in log_file_dict:
        if log['Instance_id'] == instance_id:
            log = log.update({'Instance_id': instance_id , 'start_datetime': start_datetime, 'end_datetime' : str(datetime.now(SGT))})

    df = pd.DataFrame(log_file_dict)
    df = df.to_csv('/tmp/log.csv', index = False)
    s3.meta.client.upload_file('/tmp/log.csv', bucket, 'logs/' + str(app_name) + '/' + str(app_name) + '_logs.csv')
    print('log File updated sucessfully')
    
def sns_msg(app_name, inst_id, type, e=None, logs=None):
    subject = str('Spot Event')
    if type == 'Client Error' or 'Error':
        message = '''Error Messge from spot-pipeline, PFB the status:

            App Name : {0}
            Instance ID : {1}
            Error Type : {2}
            Status : {3}
            logs : {4}'''.format(app_name, inst_id, type, e, logs)
    
    if type == 'triggered':
        message = '''Spot Interruption trigger received from spot-pipeline, PFB the status:

            App Name : {0}
            Instance ID : {1}
            Status : {2}'''.format(app_name, inst_id, type)   
            
    mail = sns.publish(TopicArn = sns_topic_arn, Message = message, Subject = subject)

def lambda_handler(event, context):
    instance_id = event['instance_id']

    try:
        csv_obj = s3_client.get_object(Bucket = bucket, Key = 'config/spot_instance_config.csv')
        csv_string = (csv_obj['Body']).read().decode('utf-8')
        config = pd.read_csv(StringIO(csv_string))

        # Step-1 : Getting all variables from config file
        config_dict = config.to_dict('records')
        for row in config_dict:
            if row['instance_id'] == instance_id:
                app_name = row['application_name']
                start_datetime = row['start_datetime']
                region = row['region_name']
                spot_request_id = row['spot_request_id']
                InstanceType_list = eval(row['instancetype_list'])
                availability_zone = row['availability_zone']
                ebs_volums = eval(row['ebs_volums'])
                security_group = eval(row['security_group_Id'])
                interuption_behavior = row['instance_interuption_behavior']
                ami = row['ami_image_id']
                od_instance_type = row['onDemand_instance_type']
                instance_state = row['instance_state']
                virtual_name = row['application_name']
                private_ip = row['private_ip']
                vpc = row['vpc']
                subnet_id = row['subnet']
                iam_role_arn = row['iam_role_arn']
                iam_role_name = row['iam_role_name']
                key_pair_name = row['key_pair_name']
                tags = eval(row['tags'])
                eni = row['eni']

                sns_msg(app_name, instance_id, 'triggered')
                inst_id = instance_id
                if instance_id != 'not_started':
                    log_csv_obj = s3_client.get_object(Bucket = bucket, Key = 'logs/' + str(app_name) + '/' + str(app_name) + '_logs.csv')
                    log_csv_string = (log_csv_obj['Body']).read().decode('utf-8')
                    log_file = pd.read_csv(StringIO(log_csv_string))
                    log_file_dict = log_file.to_dict('records')

        # Step-2 : Cancel the existing spot instance requests and terminating existing spot instance
        if instance_id != 'not_started':
            terminate_instances(instance_id, start_datetime, log_file_dict, app_name, bucket)
            print('EC2 instance termination initialized sucessfully.')
        if instance_state == 'spot':
            cancel_spot_request(spot_request_id)
            print('Existing Spot Request Cancel initiated.')

        # Step-3 : Initiating Spot instance hunt
        print('Spot Instance hunt started..')
        hunt_state = 'started'
        for InstanceType in InstanceType_list:
            if hunt_state == 'started' or hunt_state == 'no':
                print('\nSearching spot instance type : ', InstanceType)
                response = spot_init(InstanceType, iam_role_arn, iam_role_name, ami, key_pair_name, availability_zone, interuption_behavior, tags, eni)
                hunt_state = response[0]
                SpotInstanceRequestId = response[1]
                inst_id = response[2]

            if hunt_state == 'present':
                if instance_id == 'not_started':
                    df = pd.DataFrame([{'Instance_id': inst_id , 'start_datetime': str(datetime.now(SGT)), 'end_datetime' : ''}])
                    df = df.to_csv('/tmp/log.csv', index = False)
                    s3.meta.client.upload_file('/tmp/log.csv', bucket, 'logs/' + str(app_name) + '/' + str(app_name) + '_logs.csv')
                    print('log File created sucessfully')
                else:
                    log_file_dict.append({'Instance_id': inst_id , 'start_datetime': str(datetime.now(SGT)), 'end_datetime' : ''})
                    df = pd.DataFrame(log_file_dict)
                    df = df.to_csv('/tmp/log.csv', index = False)
                    s3.meta.client.upload_file('/tmp/log.csv', bucket, 'logs/' + str(app_name) + '/' + str(app_name) + '_logs.csv')
                    print('log File updated sucessfully')
                    
                stop_instance(inst_id)
                ebs_vol_id = get_ebs_vol_id(inst_id)
                volume_detach(inst_id, ebs_vol_id)
                attach_volume(inst_id, ebs_volums)
                delete_volume(ebs_vol_id)
                describe_spot_req(SpotInstanceRequestId)
                start_instance(inst_id)

                for row in config_dict:
                    if row['instance_id'] == instance_id:
                        row.update({'instance_id' : inst_id, 'spot_request_id' : SpotInstanceRequestId, 'instance_state' : 'spot','event_rule_name' : '','start_datetime': str(datetime.now(SGT))})
                df = pd.DataFrame(config_dict)
                df = df.to_csv('/tmp/spot_instance_config.csv', index = False)
                s3.meta.client.upload_file('/tmp/spot_instance_config.csv', bucket, 'config/spot_instance_config.csv')
                print('Config File updated sucessfully')
                break
        
        if hunt_state == 'no':
            print('\nOn-Demand hunt started..')
            print('Searching On-Demand instance type : ', od_instance_type)

            inst_id = ondemand_init(virtual_name, ami, od_instance_type, availability_zone, key_pair_name, iam_role_arn, iam_role_name, tags, eni)
            # Updating log file in s3
            if instance_id == 'not_started':
                df = pd.DataFrame([{'Instance_id': inst_id , 'start_datetime': str(datetime.now(SGT)), 'end_datetime' : ''}])
                df = df.to_csv('/tmp/log.csv', index = False)
                s3.meta.client.upload_file('/tmp/log.csv', bucket, 'logs/' + str(app_name) + '/' + str(app_name) + '_logs.csv')
                print('log File created sucessfully')
            else:
                log_file_dict.append({'Instance_id': inst_id , 'start_datetime': str(datetime.now(SGT)), 'end_datetime' : ''})
                df = pd.DataFrame(log_file_dict)
                df = df.to_csv('/tmp/log.csv', index = False)
                s3.meta.client.upload_file('/tmp/log.csv', bucket, 'logs/' + str(app_name) + '/' + str(app_name) + '_logs.csv')
                print('log File updated sucessfully')
                
            stop_instance(inst_id)
            ebs_vol_id = get_ebs_vol_id(inst_id)
            volume_detach(inst_id, ebs_vol_id)
            attach_volume(inst_id, ebs_volums)
            delete_volume(ebs_vol_id)
            describe_spot_req(SpotInstanceRequestId)
            start_instance(inst_id)

            rulename = ('SpotTest_' + str(datetime.now(SGT)).replace('-','').replace(':','').replace(' ','').replace('.','').split('+')[0])
            lambda_RoleARN = 'arn:aws:lambda:ap-southeast-1:263975812908:function:spot-config-tester'
            create_eventrule(rulename, lambda_RoleARN, inst_id)

            for row in config_dict:
                if row['instance_id'] == instance_id:
                    row.update({'instance_id' : inst_id, 'spot_request_id' : 'on-demand',  'instance_state' : 'on-demand', 'event_rule_name' : str(rulename),'start_datetime': str(datetime.now(SGT))})

            df = pd.DataFrame(config_dict)
            df = df.to_csv('/tmp/spot_instance_config.csv', index = False)
            s3.meta.client.upload_file('/tmp/spot_instance_config.csv', bucket, 'config/spot_instance_config.csv')
            print('Config File updated sucessfully')
            
    except ClientError as e:
        print("Client Error: ", e)
        print('Exiting with error!!!')
        sns_msg(app_name, inst_id, 'Client Error', e)

    except Exception as e:
        print("Error: ", str(e))
        print('Exiting with error!!!')
        sns_msg(app_name, inst_id, 'Error', e)

# Lambda - 2

In [ ]:
import json
import boto3
import pandas as pd
from datetime import *
from io import StringIO
from botocore.exceptions import ClientError
import time
import pytz
  
SGT = pytz.timezone('Asia/Singapore')
bucket = 'ag-nonprd-spot-config'
sns_topic_arn = 'arn:aws:sns:ap-southeast-1:263975812908:Spot-Notification'

client = boto3.client('ec2')
ec2 = boto3.resource('ec2')
evnt_client = boto3.client('events')
lambda_client = boto3.client('lambda')
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
sns = boto3.client('sns')

def spot_avail_tester(InstanceType, iam_role_arn, iam_role_name, ami, key_pair_name, availability_zone, interuption_behavior, tags, sec_eni):
    spot_create_response = client.request_spot_instances(
        InstanceCount=1,
        LaunchSpecification={
            'EbsOptimized': True,
            'IamInstanceProfile': {'Name': iam_role_name},
            'ImageId': ami,
            'InstanceType': InstanceType,
            'KeyName': key_pair_name,
            'Monitoring': {'Enabled': True}, 
            'NetworkInterfaces': [{'DeviceIndex': 0,'NetworkInterfaceId': sec_eni}],
            'Placement': {'AvailabilityZone': availability_zone}},
        Type = 'persistent',
        TagSpecifications=[{'ResourceType': 'spot-instances-request', 'Tags': tags}],
        InstanceInterruptionBehavior = interuption_behavior
    )

    SpotInstanceRequestId_t = str(spot_create_response['SpotInstanceRequests'][0]['SpotInstanceRequestId'])
    print('Test_Spot Instance Request_Id : ', SpotInstanceRequestId_t)

    time.sleep(2)
    request_response = client.describe_spot_instance_requests(SpotInstanceRequestIds = [SpotInstanceRequestId_t])
    while request_response['SpotInstanceRequests'][0]['State'] != 'active':
        request_response = client.describe_spot_instance_requests(SpotInstanceRequestIds = [SpotInstanceRequestId_t])
        if request_response['SpotInstanceRequests'][0]['State'] == 'failed':
            print('Requested Instance Type is not available')
            hunt_state = 'no'
            return [hunt_state, '', '']

    if request_response['SpotInstanceRequests'][0]['State'] == 'active':
        inst_id_t = request_response['SpotInstanceRequests'][0]['InstanceId']
        cancel_spot_request_response = client.cancel_spot_instance_requests(SpotInstanceRequestIds=[SpotInstanceRequestId_t])
        print('Test Spot Request Cancelled.')
        response = client.terminate_instances(InstanceIds=[inst_id_t])
        print('Test Spot Machine terminated.')
        hunt_state = 'present'
        return [hunt_state, '', '']

def spot_init(InstanceType, iam_role_arn, iam_role_name, ami, key_pair_name, availability_zone, interuption_behavior, tags, eni):
    spot_create_response = client.request_spot_instances(
        InstanceCount=1,
        LaunchSpecification={
            'EbsOptimized': True,
            'IamInstanceProfile': {'Name': iam_role_name},
            'ImageId': ami,
            'InstanceType': InstanceType,
            'KeyName': key_pair_name,
            'Monitoring': {'Enabled': True}, 
            'NetworkInterfaces': [{'DeviceIndex': 0,'NetworkInterfaceId': eni}],
            'Placement': {'AvailabilityZone': availability_zone}},
        Type = 'persistent',
        TagSpecifications=[{'ResourceType': 'spot-instances-request', 'Tags': tags}],
        InstanceInterruptionBehavior = interuption_behavior
    )

    SpotInstanceRequestId = str(spot_create_response['SpotInstanceRequests'][0]['SpotInstanceRequestId'])
    print('Spot Instance Request_Id : ', SpotInstanceRequestId)

    time.sleep(2)
    request_response = client.describe_spot_instance_requests(SpotInstanceRequestIds = [SpotInstanceRequestId])
    while request_response['SpotInstanceRequests'][0]['State'] != 'active':
        request_response = client.describe_spot_instance_requests(SpotInstanceRequestIds = [SpotInstanceRequestId])
        if request_response['SpotInstanceRequests'][0]['State'] == 'failed':
            print('Requested Instance Type is not available')
            hunt_state = 'no'
            return [hunt_state, SpotInstanceRequestId, '']

    if request_response['SpotInstanceRequests'][0]['State'] == 'active':
        inst_id = request_response['SpotInstanceRequests'][0]['InstanceId']
        hunt_state = 'present'

        print('Spot Instance Request Status : ', request_response['SpotInstanceRequests'][0]['State'])
        print('Spot Instance ID : ', inst_id)

        instance_response = client.describe_instance_status(InstanceIds=[inst_id])
        while instance_response['InstanceStatuses'] == []:
            instance_response = client.describe_instance_status(InstanceIds=[inst_id])
        print('Instance Status : ', instance_response['InstanceStatuses'][0]['InstanceState']['Name'], 'and Initializing...')
        while instance_response['InstanceStatuses'][0]['InstanceStatus']['Details'][0]['Status'] != 'passed' and instance_response['InstanceStatuses'][0]['SystemStatus']['Details'][0]['Status'] != 'passed':
            instance_response = client.describe_instance_status(InstanceIds=[inst_id])
        print('Instance Status : started')
        ec2.create_tags(Resources=[inst_id], Tags = tags)
        print('Tag Status : updated')
        return [hunt_state, SpotInstanceRequestId , inst_id]

def stop_instance(inst_id):
    stop_response = client.stop_instances(InstanceIds=[inst_id])
    print('EC2 Instance temporarily stop initiated..')
    spot_inst_status = client.describe_instances(InstanceIds=[inst_id])
    while spot_inst_status['Reservations'][0]['Instances'][0]['State']['Name'] != 'stopped':
        spot_inst_status = client.describe_instances(InstanceIds=[inst_id])
    print('EC2 Instance temporarily stopped.')

def get_ebs_vol_id(inst_id):
    ebs_volume_response = client.describe_instance_attribute(InstanceId = inst_id, Attribute='blockDeviceMapping')
    while ebs_volume_response['BlockDeviceMappings'][0]['Ebs']['Status'] != 'attached':
        ebs_volume_response = client.describe_instance_attribute(InstanceId = inst_id, Attribute='blockDeviceMapping')
    ebs_vol_id = ebs_volume_response['BlockDeviceMappings'][0]['Ebs']['VolumeId']
    print('EBS Volume ID : ', ebs_vol_id)
    return ebs_vol_id
    
def volume_detach(inst_id, ebs_vol_id):
    vol_detach_response = client.detach_volume(Force = True, InstanceId = inst_id, VolumeId = ebs_vol_id)
    print('EC2 Instance volume is detached..')

    vol_status = client.describe_volumes(VolumeIds=[ebs_vol_id])
    while vol_status['Volumes'][0]['State'] != 'available':
        vol_status = client.describe_volumes(VolumeIds=[ebs_vol_id])

def attach_volume(inst_id, ebs_volums):
    vol_list = []
    for vol in ebs_volums:
        response = client.attach_volume(Device = vol['device_name'], InstanceId = inst_id, VolumeId = vol['volume_id'])
        vol_list.append(vol['volume_id'])

    vol_status = client.describe_volumes(VolumeIds = vol_list)
    while vol_status['Volumes'][0]['Attachments'][0]['State'] != 'attached' and vol_status['Volumes'][1]['Attachments'][0]['State'] != 'attached':
        vol_status = client.describe_volumes(VolumeIds = vol_list)
    print('Volumes attached sucessfully..')

def delete_volume(ebs_vol_id):
    response = client.delete_volume(VolumeId = ebs_vol_id)
    print('New EBS volume delete request cretaed sucessfully.')

def describe_spot_req(SpotInstanceRequestId):
        request_response = client.describe_spot_instance_requests(SpotInstanceRequestIds = [SpotInstanceRequestId])
        while request_response['SpotInstanceRequests'][0]['State'] == 'active':
            request_response = client.describe_spot_instance_requests(SpotInstanceRequestIds = [SpotInstanceRequestId])

def start_instance(inst_id):
    spot_start_response = client.start_instances(InstanceIds=[inst_id])
    print('EC2 Instance Start initiated..')
    spot_inst_status = client.describe_instance_status(InstanceIds=[inst_id])
    while spot_inst_status['InstanceStatuses'] == []:
        spot_inst_status = client.describe_instance_status(InstanceIds=[inst_id])
    print('EC2 Instance Status : ', spot_inst_status['InstanceStatuses'][0]['InstanceState']['Name'], 'and Initializing...')
    
def terminate_instances(instance_id, start_datetime, log_file_dict, app_name, bucket):
    response = client.terminate_instances(InstanceIds=[instance_id])
    inst_status = client.describe_instances(InstanceIds=[instance_id])
    while inst_status['Reservations'][0]['Instances'][0]['State']['Name'] != 'terminated':
        inst_status = client.describe_instances(InstanceIds=[instance_id])
    print('Ondemand Instance terminated sucessfully')
    for log in log_file_dict:
        if log['Instance_id'] == instance_id:
            log = log.update({'Instance_id': instance_id , 'start_datetime': start_datetime, 'end_datetime' : str(datetime.now(SGT))})

    df = pd.DataFrame(log_file_dict)
    df = df.to_csv('/tmp/log.csv', index = False)
    s3.meta.client.upload_file('/tmp/log.csv', bucket, 'logs/' + str(app_name) + '/' + str(app_name) + '_logs.csv')
    print('log File updated sucessfully')
    
def vol_status(ebs_volums):
    ebs_volums_list = []
    for vol in ebs_volums:
        ebs_volums_list.append(vol['volume_id'])
        
    vol_status = client.describe_volumes(VolumeIds = ebs_volums_list)
    while vol_status['Volumes'][0]['State'] != 'available':
        vol_status = client.describe_volumes(VolumeIds = ebs_volums_list)
    print('Both volumes available now.')

def sns_msg(app_name, inst_id, type, e=None, logs=None):
    subject = str('Spot Event')
    if type == 'Client Error' or 'Error':
        message = '''Error Messge from spot-pipeline, PFB the status:
            
            Lambda Function : spot-config-tester (lambda2)
            App Name : {0}
            Instance ID : {1}
            Error Type : {2}
            Status : {3}
            logs : {4}'''.format(app_name, inst_id, type, e, logs)
            
    mail = sns.publish(TopicArn = sns_topic_arn, Message = message, Subject = subject)

def lambda_handler(event, context):
    try:
        instance_id = event['instance_id']
        inst_id = instance_id
    
        csv_obj = s3_client.get_object(Bucket = bucket, Key = 'config/spot_instance_config.csv')
        csv_string = (csv_obj['Body']).read().decode('utf-8')
        config = pd.read_csv(StringIO(csv_string))
    
        # Step-1 : Getting all variables from config file
        config_dict = config.to_dict('records')
        for row in config_dict:
            if row['instance_id'] == instance_id:
                app_name = row['application_name']
                start_datetime = row['start_datetime']
                region = row['region_name']
                spot_request_id = row['spot_request_id']
                InstanceType_list = eval(row['instancetype_list'])
                availability_zone = row['availability_zone']
                ebs_volums = eval(row['ebs_volums'])
                security_group = eval(row['security_group_Id'])
                interuption_behavior = row['instance_interuption_behavior']
                ami = row['ami_image_id']
                od_instance_type = row['onDemand_instance_type']
                instance_state = row['instance_state']
                virtual_name = row['application_name']
                private_ip = row['private_ip']
                vpc = row['vpc']
                subnet_id = row['subnet']
                iam_role_arn = row['iam_role_arn']
                iam_role_name = row['iam_role_name']
                key_pair_name = row['key_pair_name']
                tags = eval(row['tags'])
                eni = row['eni']
                event_rule_name = row['event_rule_name']
                sec_eni = row['sec_eni']
                
                if instance_id != 'not_started':
                    log_csv_obj = s3_client.get_object(Bucket = bucket, Key = 'logs/' + str(app_name) + '/' + str(app_name) + '_logs.csv')
                    log_csv_string = (log_csv_obj['Body']).read().decode('utf-8')
                    log_file = pd.read_csv(StringIO(log_csv_string))
                    log_file_dict = log_file.to_dict('records')
            
        print('Spot Instance hunt started..')
        hunt_state = 'started'
        for InstanceType in InstanceType_list:
            if hunt_state == 'started' or hunt_state == 'no':
                print('\nSearching spot instance type : ', InstanceType)
                response = spot_avail_tester(InstanceType, iam_role_arn, iam_role_name, ami, key_pair_name, availability_zone, interuption_behavior, tags, sec_eni)
                hunt_state = response[0]
                
            if hunt_state == 'present':
                
                terminate_instances(instance_id, start_datetime, log_file_dict, app_name, bucket)
                
                response = spot_init(InstanceType, iam_role_arn, iam_role_name, ami, key_pair_name, availability_zone, interuption_behavior, tags, eni)
                hunt_state = response[0]
                SpotInstanceRequestId = response[1]
                inst_id = response[2]
                
                start_time = str(datetime.now(SGT))
                log_file_dict.append({'Instance_id': inst_id , 'start_datetime': start_time, 'end_datetime' : ''})
                df = pd.DataFrame(log_file_dict)
                df = df.to_csv('/tmp/log.csv', index = False)
                s3.meta.client.upload_file('/tmp/log.csv', bucket, 'logs/' + str(app_name) + '/' + str(app_name) + '_logs.csv')
                print('log File updated sucessfully')
            
            if hunt_state == 'present':
                vol_status(ebs_volums)
                stop_instance(inst_id)
                ebs_vol_id = get_ebs_vol_id(inst_id)
                volume_detach(inst_id, ebs_vol_id)
                attach_volume(inst_id, ebs_volums)
                delete_volume(ebs_vol_id)
                describe_spot_req(SpotInstanceRequestId)
                start_instance(inst_id)

                response = evnt_client.disable_rule(Name = event_rule_name)
                print('Event Rule is Disabled.')
                response = evnt_client.remove_targets(Rule = event_rule_name,Ids=['abc'], Force=True)
                print('Event rule target is deleted sucessfully.')
                response = evnt_client.delete_rule(Name = event_rule_name, Force = True)
                print('Event rule is deleted sucessfully.')
            
                for row in config_dict:
                    if row['instance_id'] == instance_id:
                        row.update({'instance_id' : inst_id, 'spot_request_id' : SpotInstanceRequestId, 'instance_state' : 'spot','event_rule_name' : '','start_datetime': start_time})
                df = pd.DataFrame(config_dict)
                df = df.to_csv('/tmp/spot_instance_config.csv', index = False)
                s3.meta.client.upload_file('/tmp/spot_instance_config.csv', bucket, 'config/spot_instance_config.csv')
                print('Config File updated sucessfully')
                break
        
        if hunt_state == 'no':
            print('Spot instances are not free yet')
            
    except ClientError as e:
        print("Client Error: ", e)
        print('Exiting with error!!!')
        sns_msg(app_name, inst_id, 'Client Error', e)

    except Exception as e:
        print("Error: ", str(e))
        print('Exiting with error!!!')
        sns_msg(app_name, inst_id, 'Error', e)
